In [1]:
from pymatgen.io.vasp import Poscar, Xdatcar
from pymatgen.symmetry.groups import SpaceGroup
from pymatgen import Structure, Lattice
import numpy as np
import operator
from site_analysis import Polyhedron, Atom, Analysis, get_vertex_indices, AtomsTrajectory, SitesTrajectory
from polyhedral_analysis.polyhedra_recipe import matching_site_indices
from collections import Counter

In [2]:
all_na_structure = Poscar.from_file('../na_sn_all_na_ext.POSCAR.vasp').structure
vertex_species = 'S'
centre_species = 'Na'

sg = SpaceGroup('I41/acd:2')
lattice = all_na_structure.lattice
na1 = Structure.from_spacegroup(sg='I41/acd:2', lattice=lattice, species=['Na'], coords=[[0.25, 0.0, 0.125]])
na2 = Structure.from_spacegroup(sg='I41/acd:2', lattice=lattice, species=['Na'], coords=[[0.00, 0.0, 0.125]])
na3 = Structure.from_spacegroup(sg='I41/acd:2', lattice=lattice, species=['Na'], coords=[[0.0, 0.25, 0.0]])
na4 = Structure.from_spacegroup(sg='I41/acd:2', lattice=lattice, species=['Na'], coords=[[0.0, 0.0, 0.0]])
na5 = Structure.from_spacegroup(sg='I41/acd:2', lattice=lattice, species=['Na'], coords=[[0.75, 0.25, 0.0]])
na6 = Structure.from_spacegroup(sg='I41/acd:2', lattice=lattice, species=['Na'], coords=[[0.5, 0.75, 0.625]])
i2  = Structure.from_spacegroup(sg='I41/acd:2', lattice=lattice, species=['Na'], coords=[[0.666, 0.1376, 0.05]])
na_structures = {'Na1': na1,
                 'Na2': na2,
                 'Na3': na3,
                 'Na4': na4,
                 'Na5': na5,
                 'Na6': na6, 
                 'i2': i2}

s = Structure.from_spacegroup(sg='I41/acd:2', lattice=lattice, 
                          species=['Sn','S','S','S', 'P','Na','Na','Na','Na','Na','K','Li','Mg'], 
                          coords=[[0.25, 0.0, 0.25],
                                  [0.326, 0.8997, 0.1987],
                                  [0.1258, 0.8972, 0.2999],
                                  [0.4097, 0.8339, 0.332],
                                  [0, 0.75, 0.125],
                                  [0.25, 0.0, 0.125],
                                  [0.0, 0.0, 0.125],
                                  [0.0, 0.25, 0.0],
                                  [0.0, 0.0, 0.0],
                                  [0.75, 0.25, 0.0],
                                  [0.5, 0.75, 0.625],
                                  [0.65, 0.375, 0.05],
                                  [0.25000,  0.75000,  0.00000]]) 

s.to(filename='all_atoms_ge.cif')

labels = {}
for l, structure in na_structures.items():
    indices = matching_site_indices(all_na_structure, structure)
    for i in indices:
        labels[i] = l
sorted_labels = [ labels[i] for i in sorted(labels) ]

In [3]:
# Ge
# find atom indices (within species) for all polyhedra vertex atoms
vertex_indices = get_vertex_indices(all_na_structure, centre_species=centre_species, 
                                    vertex_species=vertex_species, cutoff=4.3)

structure = Poscar.from_file('./POSCAR').structure
# create Polyhedron objects
polyhedra = [Polyhedron(vertex_species=vertex_species, vertex_indices=vi, label=label) 
             for vi, label in zip(vertex_indices, sorted_labels)]
# create Atom objects
atoms = [Atom(species_string=centre_species) for site in structure if site.species_string is 'Na']
analysis = Analysis(polyhedra, atoms)

xdatcar = Xdatcar('./XDATCAR')
for timestep, s in enumerate(xdatcar.structures):
    analysis.append_timestep(s, t=timestep)

a = Counter()
b = Counter()
for n,i in enumerate(analysis.st):
    a += Counter([ l for d,l  in zip(analysis.st[n], analysis.site_labels()) if len(d)>0])
    b += Counter([ l for d,l  in zip(analysis.st[n], analysis.site_labels())])

for i in a:
    print(i)
    print(a[i] / b[i])

Na3
0.7625
Na4
0.959375
Na2
0.93125
Na5
0.95
Na1
0.959375
i2
0.003125


In [ ]:
t = 0
for u, o in a.items():
    print(u, o / b[u] * Counter([ l for d,l  in zip(analysis.st[0], analysis.site_labels())])[u])
    t += o / b[u] * Counter([ l for d,l  in zip(analysis.st[0], analysis.site_labels())])[u]
print(t)

In [ ]:
polyhedra[0].as_dict()

In [ ]:
analysis.reset()
xdatcar = Xdatcar('XDATCAR')
analysis.trajectory_from_structures( xdatcar.structures, progress='notebook')

In [ ]:
n_timesteps = len(analysis.timesteps)
c_sites = Counter(analysis.site_labels())
c = Counter()
p_occ = {}
for site in analysis.sites:
    c[site.label] += len([ 1 for ts in site.trajectory if len(ts)>0 ])
for k, v in c.items():
    p_occ[k] = v / c_sites[k] / n_timesteps
p_occ

In [ ]:
structure

In [ ]:
for p in polyhedra:
    print(p.label)

In [ ]:
for p in polyhedra:
    print(p.as_dict())

In [ ]:
print(len(polyhedra))

In [ ]:
len(vertex_indices)